In [1]:
"""
introducing tensoflow functionality: graph, scope, summary, and checkpoints

tensorflow v1.0.0 solving xor
"""

import numpy as np

data = np.array([
        [0,0],
        [0,1],
        [1,0],
        [1,1],
    ])

labels = np.array([
        [0],
        [1],
        [1],
        [0],
    ])

train_data = data
train_labels = labels
test_data = data
test_labels = labels

input_dim = data.shape[1]
output_dim = labels.shape[1]
hidden_dim = 4


In [2]:
from params import get_FLAGS

FLAGS = get_FLAGS()
# print("Parameters:\n")
# for attr, value in sorted(FLAGS.__flags.items()):
#     print("{}={}".format(attr, value))

In [3]:
import tensorflow as tf
import datetime
import time
import os

with tf.Graph().as_default() as g:

    # ----------------------
    # MLP
    # ----------------------
    with tf.device('/cpu:0'), tf.name_scope('mlp'):
        x = tf.placeholder(tf.float32, [None, input_dim], name='input')
        y_ = tf.placeholder(tf.float32, [None, output_dim],  name='label')

        w1 = tf.Variable(tf.random_uniform([input_dim, hidden_dim], minval=-0.9, maxval=0.9), name='w1')
        b1 = tf.Variable(tf.random_uniform([hidden_dim], minval=-0.9, maxval=0.9), name='b1')

        hidden_layer = tf.sigmoid(tf.matmul(x, w1) + b1)

        w2 = tf.Variable(tf.random_uniform([hidden_dim, output_dim], minval=-0.9, maxval=0.9), name='w2')
        b2 = tf.Variable(tf.random_uniform([output_dim], minval=-0.9, maxval=0.9), name='b2')

        y = tf.sigmoid(tf.matmul(hidden_layer, w2) + b2)

        
    # ----------------------
    # Training
    # ----------------------
    with tf.device('/gpu:0'), tf.name_scope('train'):
        global_step = tf.Variable(0, name="global_step", trainable=False)
        cross_entropy = tf.reduce_mean(- y_ * tf.log(y) - (1.0-y_) * tf.log(1.0-y), name="ce")
        optimizer = tf.train.GradientDescentOptimizer(0.1)
        g_and_v = optimizer.compute_gradients(cross_entropy)
        train_step = optimizer.apply_gradients(g_and_v, global_step=global_step)
        
        correct_prediction = tf.equal(tf.round(y), tf.round(y_))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
    
    
    # ----------------------
    # Session
    # ----------------------
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)

    sess = tf.Session(config=session_conf)
    
    with sess.as_default():

        # ----------------------
        # Output folders
        # ----------------------
        time_str = datetime.datetime.now().isoformat().replace(":",".") #windows does not allow :
        pwd = os.path.abspath(os.path.join(os.path.curdir))
        out_dir = os.path.join(pwd, 'runs', time_str)
        
        # summary folders
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        
        #
        checkpoint_dir = os.path.join(out_dir, "checkpoints")
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        
        # saver need an exisiting folder to save, but summary filewriter dont
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        
        

        # ----------------------
        # Summaries
        # ----------------------
        loss_summary = tf.summary.scalar('loss', cross_entropy)
        acc_summary = tf.summary.scalar('accuracy', accuracy)
        gradient_summary = []
        for g, v in g_and_v:
            if g is not None:
                sparsity = tf.nn.zero_fraction(g)
                g_hist_summary = tf.summary.histogram('{}/grad/hist'.format(v.name.replace(':','_')), g)
                g_sparsity_summary = tf.summary.scalar('{}/grad/sparsity'.format(v.name.replace(':','_')), sparsity)
                gradient_summary.append(g_hist_summary)
                gradient_summary.append(g_sparsity_summary)
        g_summary_merged = tf.summary.merge(gradient_summary)
        
        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, g_summary_merged])
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
    
    
        # ----------------------
        # Saver
        # ----------------------
        saver = tf.train.Saver(tf.global_variables())
    
        sess.run(tf.global_variables_initializer())

        for i in range(FLAGS.num_epochs + 1):
            
            feed_dict= {
                x: train_data, y_: train_labels
            }
            _, ce, prediction = sess.run([train_step, cross_entropy, tf.round(y)], feed_dict=feed_dict)
            current_step = tf.train.global_step(sess, global_step)
            
            if (i%100)==0:
                print('current step', current_step)
                print('Iteration: ', i)
                print('予測: ')
                print('    ', prediction)
                print('誤差= ', ce)
                print()
                
                feed_dict = {
                  x: test_data,
                  y_: test_labels
                }
                
                step, d_summaries, t_summaries = sess.run(
                    [global_step, dev_summary_op, train_summary_op],
                    feed_dict=feed_dict
                )
                dev_summary_writer.add_summary(d_summaries, step)
                train_summary_writer.add_summary(t_summaries, step)
                
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))
                

current step 1
Iteration:  0
予測: 
     [[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]]
誤差=  0.77118

Saved model checkpoint to C:\Users\ray\Dropbox\github\XOR-gate-tensorflow-mechanics\runs\2017-03-28T06.20.36.243445\checkpoints\model-1

current step 101
Iteration:  100
予測: 
     [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]]
誤差=  0.692562

Saved model checkpoint to C:\Users\ray\Dropbox\github\XOR-gate-tensorflow-mechanics\runs\2017-03-28T06.20.36.243445\checkpoints\model-101

